In [12]:
%cd ~/change
import pickle
import torch
from transformers import AutoTokenizer,BertForMaskedLM,Trainer,TrainingArguments
with open('data/mclose.pickle',mode='rb') as f:
    train_x = pickle.load(f)
    train_y = pickle.load(f)

/home/sibava/change


In [13]:
tokenizer = AutoTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-v2')
model = BertForMaskedLM.from_pretrained('cl-tohoku/bert-base-japanese-v2')
model_path = 'models/select_trained2.pth'
model.load_state_dict(torch.load(model_path))

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
kaku_dict ={
    'から':['から','が'],
    'が':['を'],
    'で':['が','で'],
    'と':['と'],
    'に':['が','に'],
    'の':['を'],
    'へ':['へ'],
    'まで':['まで'],
    'を':['を'],
    'o':['o']
    }
token_dict = {}
kakuid_dict = {}
for k,v in kaku_dict.items():
    kakuid_dict[tokenizer.encode(k,add_special_tokens=False)[0]] = tokenizer.encode(v,add_special_tokens=False)
del kakuid_dict[912]
kakuid_dict[11172] = [11172]
for k,v in kaku_dict.items():
    token_dict[k] = tokenizer.encode(k,add_special_tokens=False)[0]
token_dict['まで'] = 11172

In [6]:
texts = []
labels = []
before = []
after = []
for x,y in zip(train_x,train_y):
    texts.append(f'{x[0]}[SEP]{x[1]}[MASK]{x[2]}')
    labels.append(f'{x[0]}[SEP]{x[1]}{y}{x[2]}')
    before.append(token_dict[x[3]])
    after.append(y)
test_texts = tokenizer(texts,truncation=True,padding=True,return_tensors='pt')
test_labels = tokenizer(labels,truncation=True,padding=True,return_tensors='pt').input_ids

class KakuDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels, before,after):
        self.encodings = encodings
        self.labels = labels
        self.before = before
        self.after = after
    def __getitem__(self, idx):
        item = {key: torch.tensor(val)[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        item['before'] = self.before[idx]
        item['after'] = self.after[idx]
        return item
        # item = {'text':tokenizer.decode(self.encodings[idx].input_ids)}
        # item['labels'] = tokenizer.decode(self.labels[idx].input_ids) 
        # return item
        
    def __len__(self):
        return len(self.labels)

test_dataset = KakuDataset(test_texts,test_labels,before,after)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
def eval(model,dataset):
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    l = len(dataset)
    c = 0
    step = 0
    loader = torch.utils.data.DataLoader(test_dataset,batch_size=1,num_workers=6)
    model.eval()
    model.to(device)
    log = open('./log.txt','w')
    for batch in loader:
        masked_indexes = torch.where(batch['input_ids'] == tokenizer.mask_token_id)[1].tolist()
        outputs = model(input_ids=batch['input_ids'].to(device),attention_mask=batch['attention_mask'].to(device),token_type_ids=batch['token_type_ids'].to(device))
        for m in masked_indexes:
            corrects = batch['after'][0]
            # candidate_set = [[outputs['logits'][0][m],tokenizer.decode(i).replace(' ','')] for i in index_list]
            pred = tokenizer.decode(torch.max(outputs['logits'][0][m],0).indices).replace(' ','')
            if (pred in corrects):
                c += 1
            else:
                print(pred)
                print(tokenizer.decode(batch['labels'][0])) 
                # log.write(f'{pred},{tokenizer.decode(batch['labels'][0])}'')

    print(f'\n\n accuracy score  {c/l}% ')
    log.write(f'\n\n accuracy score  {c/l}%') 
    log.close()
eval(model,test_dataset) #0.944,複数選択で片方のみを正解 93.2シングル予測 95.5両方正解 95.0正式結果 95.7select訓練

/home/sibava/miniconda3/envs/kakukoutai/lib/python3.9/site-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/tmp/ipykernel_2793453/560662646.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val)[idx] for key, val in self.encodings.items()}
/tmp/ipykernel_2793453/560662646.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_

から
[CLS] 強大 な 権限 を 持つ 大蔵 省 が 諸悪 の 根源 の よう に 言わ れ 他 省庁 や 海外 から 大蔵 分割 論 が 唱え られ 外務 省 通産 省 無用 論 や 建設 省 農水 省 合併 論 郵政 省 の 民営 化 論 など が 半ば 公然 と 語ら れる よう に なり まし た [SEP] 海外 が 唱える [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
とも
[CLS] 与野党 とも 期待 や 願望 が 込め られ て いる よう だ が 村山 氏 海部 氏 どちら の 予想 が 当たる の か はたまた 予想 外 の 人物 が 登場 し て いる の だろう か — — [SEP] 与野党 に 込める よう だ [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 

In [7]:
tokenizer.decode(890)

'と'

In [11]:
def strict_eval(model,dataset):
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    l = len(dataset)
    c = 0
    step = 0
    loader = torch.utils.data.DataLoader(test_dataset,batch_size=1,num_workers=6)
    model.eval()
    model.to(device)
    log = open('./log.txt','w')
    for batch in loader:
        masked_indexes = torch.where(batch['input_ids'] == tokenizer.mask_token_id)[1].tolist()
        outputs = model(input_ids=batch['input_ids'].to(device),attention_mask=batch['attention_mask'].to(device),token_type_ids=batch['token_type_ids'].to(device))
        for m in masked_indexes:
            corrects = batch['after'][0]
            index_list = kakuid_dict[batch['before'][0].item()]
            candidate_set = [[outputs['logits'][0][m][i],tokenizer.decode(i).replace(' ','')] for i in index_list]
            candidate_set.sort(reverse=True)
            pred = candidate_set[0][1]
            if (pred in corrects):
                c += 1
            else:
                print(pred)
                print(tokenizer.decode(batch['labels'][0])) 
                # log.write(f'{pred},{tokenizer.decode(batch['labels'][0])}'')

    print(f'\n\n accuracy score  {c/l}% ')
    log.write(f'\n\n accuracy score  {c/l}%') 
    log.close()

strict_eval(model,test_dataset) #0.964open

/tmp/ipykernel_668795/2106991734.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val)[idx] for key, val in self.encodings.items()}
/tmp/ipykernel_668795/2106991734.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])
/tmp/ipykernel_668795/2106991734.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val)[idx] for key, val in self.encodings.items()}
/tmp/ipykernel_668795/2106991734.py:20: UserWarning: To copy construct from a tensor, 

で
[CLS] 同 審議 会 で 認め られる と 手続き の 期間 を 経 て 今春 に も 値上げ が 実施 さ れる 見通し [SEP] 同 審議 会 が 認める [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
で
[CLS] 今回 の 交渉 で 最大 の 焦点 だっ た 日本 の 公的 年金 資金 の 運用 問題 に つい て は 信託 銀行 を 通じ た 間接 的 な 方法 に よっ て 外資 系 を 含め た 投資 顧問 会社 の 運用 市場 参入 を 認める こと で 最終 的 に 決着 が 図ら れ た [SEP] こと が 図る [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PA